# Build quilt data package on the fly

`pip install quilt`

In [1]:
import quilt

pkg_name = 'iconix/deephypebot'

# start with an empty package
#quilt.build(pkg_name)

In [2]:
import os
import pandas as pd

BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '..', 'datasets')
reviews_f = 'reviews_and_metadata'

reviews_5yrs_f = os.path.join(DATA_DIR, f'{reviews_f}_5yrs.json')
reviews_5yrs_df = pd.read_json(reviews_5yrs_f)
print(f'num_reviews: {len(reviews_5yrs_df)}')
reviews_5yrs_df.head()

num_reviews: 25160


,artist,artist_id,audio_features,author,content,date_published,dek,desc,direction,domain,...,pages_rendered,post_title,posturl,rendered_pages,sitename,song_title,spotify_genres,spotify_id,total_pages,word_count
0,Mt. Joy,69tiO1fG8VWduDl3ji2qhI,"{'danceability': 0.486, 'energy': 0.638, 'key'...",None,New Music\nMt. Joy reached out to us with the ...,2017-11-20T13:33:10.000Z,NaN,“Silver Lining” is the fourth released single ...,ltr,www.buffablog.com,...,NaN,"Mt. Joy - ""Silver Lining""",http://www.buffablog.com/mt-joy-silver-lining/,1.0,buffaBLOG,Silver Lining,"[deep new americana, folk-pop, indie pop, mode...",2fpDrL2Vpee0JnM6AoUFvl,1,86
1,Mt. Joy,69tiO1fG8VWduDl3ji2qhI,"{'danceability': 0.486, 'energy': 0.638, 'key'...",Martin,Folk rockers Mt. Joy have debuted their new so...,2017-11-16T00:00:00.000Z,NaN,“Silver Lining” is the fourth released single ...,ltr,www.discobelle.net,...,NaN,Mt. Joy – Silver Lining | Discobelle.net,http://www.discobelle.net/2017/11/16/mt-joy-si...,1.0,Discobelle,Silver Lining,"[deep new americana, folk-pop, indie pop, mode...",2fpDrL2Vpee0JnM6AoUFvl,1,83
2,Mt. Joy,69tiO1fG8VWduDl3ji2qhI,"{'danceability': 0.486, 'energy': 0.638, 'key'...",Nasko,You know we're digging Mt. Joy.\nTheir new sin...,2017-11-14T14:30:26.000Z,NaN,“Silver Lining” is the fourth released single ...,ltr,www.stereofox.com,...,NaN,Mt. Joy - Silver Lining | Stereofox Music Blog,https://www.stereofox.com/mt-joy-silver-lining/,1.0,Stereofox,Silver Lining,"[deep new americana, folk-pop, indie pop, mode...",2fpDrL2Vpee0JnM6AoUFvl,1,75
3,Mt. Joy,69tiO1fG8VWduDl3ji2qhI,"{'danceability': 0.486, 'energy': 0.638, 'key'...",D & d,"Nothing against the profession, but the U.S. h...",2018-06-20T10:53:00.000Z,NaN,“Silver Lining” is the fourth released single ...,ltr,indieobsessive.blogspot.com,...,NaN,“Silver Lining” by Mt. Joy – A Song Review,http://indieobsessive.blogspot.com/2017/11/sil...,1.0,Indie Obsessive,Silver Lining,"[deep new americana, folk-pop, indie pop, mode...",2fpDrL2Vpee0JnM6AoUFvl,1,416
4,Opia,70zHrrrPfBkVV44AEhFyyh,"{'danceability': 0.609, 'energy': 0.623, 'key'...",Nat Morawski,Connecticut duo Opia have released a guitar he...,2018-05-24T00:00:00.000Z,NaN,\n\nI don’t know why but this feels like a bre...,ltr,acidstag.com,...,NaN,Opia – ‘Four Winds’,https://acidstag.com/2018/05/opia-four-winds/,1.0,acid stag,Four Winds,[vapor soul],3NPiANHZYahLZhUT00GwTw,1,148


In [3]:
# https://stackoverflow.com/a/38231651

# quilt cannot handle column 'audio_features' because it is a dict - convert it into multiple columns
# convert original 'audio_features' column from dict to JSON string
audio_df = reviews_5yrs_df['audio_features'].apply(pd.Series)[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
str_df = reviews_5yrs_df['audio_features'].astype('|S')
reviews_5yrs_df = pd.concat([reviews_5yrs_df.drop(['audio_features'], axis=1), str_df, audio_df], axis=1)
reviews_5yrs_df.columns

Index(['artist', 'artist_id', 'author', 'content', 'date_published', 'dek',
       'desc', 'direction', 'domain', 'error', 'excerpt', 'extract_url',
       'failed', 'genius_id', 'genres', 'itemid', 'lang', 'lead_image_url',
       'masked_content', 'messages', 'next_page_url', 'pages_rendered',
       'post_title', 'posturl', 'rendered_pages', 'sitename', 'song_title',
       'spotify_genres', 'spotify_id', 'total_pages', 'word_count',
       'audio_features', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature'],
      dtype='object')

In [4]:
from quilt.data.iconix import deephypebot

# put data in it
deephypebot._set(['reviews_and_metadata_5yrs'], reviews_5yrs_df)
deephypebot.reviews_and_metadata_5yrs()

,artist,artist_id,author,content,date_published,dek,desc,direction,domain,error,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Mt. Joy,69tiO1fG8VWduDl3ji2qhI,None,New Music\nMt. Joy reached out to us with the ...,2017-11-20T13:33:10.000Z,NaN,“Silver Lining” is the fourth released single ...,ltr,www.buffablog.com,NaN,...,-6.231,1.0,0.0323,0.000921,0.000000,0.0780,0.1780,144.458,199827.0,4.0
1,Mt. Joy,69tiO1fG8VWduDl3ji2qhI,Martin,Folk rockers Mt. Joy have debuted their new so...,2017-11-16T00:00:00.000Z,NaN,“Silver Lining” is the fourth released single ...,ltr,www.discobelle.net,NaN,...,-6.231,1.0,0.0323,0.000921,0.000000,0.0780,0.1780,144.458,199827.0,4.0
2,Mt. Joy,69tiO1fG8VWduDl3ji2qhI,Nasko,You know we're digging Mt. Joy.\nTheir new sin...,2017-11-14T14:30:26.000Z,NaN,“Silver Lining” is the fourth released single ...,ltr,www.stereofox.com,NaN,...,-6.231,1.0,0.0323,0.000921,0.000000,0.0780,0.1780,144.458,199827.0,4.0
3,Mt. Joy,69tiO1fG8VWduDl3ji2qhI,D & d,"Nothing against the profession, but the U.S. h...",2018-06-20T10:53:00.000Z,NaN,“Silver Lining” is the fourth released single ...,ltr,indieobsessive.blogspot.com,NaN,...,-6.231,1.0,0.0323,0.000921,0.000000,0.0780,0.1780,144.458,199827.0,4.0
4,Opia,70zHrrrPfBkVV44AEhFyyh,Nat Morawski,Connecticut duo Opia have released a guitar he...,2018-05-24T00:00:00.000Z,NaN,\n\nI don’t know why but this feels like a bre...,ltr,acidstag.com,NaN,...,-6.575,1.0,0.1020,0.190000,0.000034,0.0625,0.5820,87.974,223814.0,4.0
5,Opia,70zHrrrPfBkVV44AEhFyyh,Adeel Amini,Now this is inventive. Not that we'd expect an...,2018-05-17T11:30:22.000Z,NaN,\n\nI don’t know why but this feels like a bre...,ltr,pressplayok.com,NaN,...,-6.575,1.0,0.1020,0.190000,0.000034,0.0625,0.5820,87.974,223814.0,4.0
6,ELOHIM,6wKxOKEA3K6R2UZ3COLXEY,Oblivious Pop,Since the very first release from Elohim about...,2018-06-20T16:01:00.000Z,NaN,\n\n?\n\n,ltr,www.obliviouspop.com,NaN,...,-6.560,1.0,0.0347,0.139000,0.000003,0.1110,0.2990,120.046,225531.0,4.0
7,ELOHIM,6wKxOKEA3K6R2UZ3COLXEY,None,New Music\nNow this is how you get a party sta...,2018-05-09T22:44:56.000Z,NaN,\n\n?\n\n,ltr,www.buffablog.com,NaN,...,-6.560,1.0,0.0347,0.139000,0.000003,0.1110,0.2990,120.046,225531.0,4.0
8,ELOHIM,6wKxOKEA3K6R2UZ3COLXEY,None,"This Friday, April 27th, ELOHIM releases her l...",None,NaN,\n\n?\n\n,ltr,www.theautumnroses.com,NaN,...,-6.560,1.0,0.0347,0.139000,0.000003,0.1110,0.2990,120.046,225531.0,4.0
9,ELOHIM,6wKxOKEA3K6R2UZ3COLXEY,None,What a long way Elohim has come since the rele...,2018-04-24T17:12:03.000Z,NaN,\n\n?\n\n,ltr,www.highclouds.org,NaN,...,-6.560,1.0,0.0347,0.139000,0.000003,0.1110,0.2990,120.046,225531.0,4.0


In [5]:
quilt.build(pkg_name, deephypebot)

In [6]:
# log in and push to the registry
quilt.login()
quilt.push(pkg_name, is_public=True)

Launching a web browser...
If that didn't work, please visit the following URL: https://pkg.quiltdata.com/login
Failed to launch the browser: Command '['xdg-open', 'https://pkg.quiltdata.com/login']' returned non-zero exit status 3.

Enter the code from the webpage: eyJjb2RlIjogImMwZGYxNzI5LWY4YzMtNDA0MS05YmNjLWEyZjkyZjZjNmU1NyIsICJpZCI6ICJkMzI3ZTdkYS1lZDFiLTRiMWMtYTFlMC1lM2ZkNzk4NDJlMzkifQ==
Fetching upload URLs from the registry...


  0%|          | 0.00/36.8M [00:00<?, ?B/s]

Uploading 1 fragments (36757347 bytes)...


100%|██████████| 36.8M/36.8M [00:03<00:00, 12.2MB/s]


Uploading package metadata...
Updating the 'latest' tag...
Push complete. iconix/deephypebot is live:
https://quiltdata.com/package/iconix/deephypebot


In [7]:
quilt.inspect(pkg_name)

/home/ubuntu/.local/share/QuiltCli/quilt_packages/pkgs/Quilt/iconix/deephypebot
└── reviews_and_metadata_5yrs: shape (25160, 45), types 'artist': object, 'artist_id': object, 'author': object, 'conte…
